<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Очистка-текста-от-знаков-препинания" data-toc-modified-id="Очистка-текста-от-знаков-препинания-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Очистка текста от знаков препинания</a></span></li><li><span><a href="#Токенизация-и-лемматизация" data-toc-modified-id="Токенизация-и-лемматизация-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Токенизация и лемматизация</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#DecisionTreeClassifier" data-toc-modified-id="DecisionTreeClassifier-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>DecisionTreeClassifier</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для портала

## Подготовка

In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import wordnet 
nltk.download('wordnet')
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import f1_score
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.tree import DecisionTreeClassifier

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv(...)
df

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0


Видим большой обьем текстовых данных. Если работать со всем датасетом, то мой ПК не вывозит обучение и предсказание модели,недостаточно RAM. А в тренажере яндекса постоянно падает ядро, поэтому я вынуждена выделить из датасета часть данных и показать работу на них.

Рассмотрим сколько всего негативных комментариев. 

In [3]:
df[df['toxic'] == 1].count()

text     16225
toxic    16225
dtype: int64

Возьмем все строки из негативного класса 1, в количестве 16225. А так же возьмем 15% от похитивного класса 0 - их количество 21500.

Далее,склеим их в датасет и произведем предобработку данных.

In [4]:
df_neg = df[df['toxic'] == 1]
df_neg

,text,toxic
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
12,Hey... what is it..\n@ | talk .\nWhat is it......,1
16,"Bye! \n\nDon't look, come or think of comming ...",1
42,You are gay or antisemmitian? \n\nArchangel WH...,1
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1
...,...,...
159494,"""\n\n our previous conversation \n\nyou fuckin...",1
159514,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1
159541,Your absurd edits \n\nYour absurd edits on gre...,1
159546,"""\n\nHey listen don't you ever!!!! Delete my e...",1


In [5]:
df_pos = df[df['toxic'] == 0][:21500]
df_pos

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
23971,"""\n\n This is wrong \n\n""""Change in playoff fo...",0
23972,"The article Fisher, Jonothan has been speedily...",0
23973,"""\nYour bizarre fascination with me is disturb...",0
23974,Minnie Minoso's first game with White Sox (hom...,0


In [6]:
df_slice = pd.concat([df_neg, df_pos], ignore_index=True)
df_slice

,text,toxic
0,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
1,Hey... what is it..\n@ | talk .\nWhat is it......,1
2,"Bye! \n\nDon't look, come or think of comming ...",1
3,You are gay or antisemmitian? \n\nArchangel WH...,1
4,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1
...,...,...
37720,"""\n\n This is wrong \n\n""""Change in playoff fo...",0
37721,"The article Fisher, Jonothan has been speedily...",0
37722,"""\nYour bizarre fascination with me is disturb...",0
37723,Minnie Minoso's first game with White Sox (hom...,0


In [7]:
df_slice['toxic'].value_counts(normalize=True) 

0    0.569914
1    0.430086
Name: toxic, dtype: float64

### Очистка текста от знаков препинания

С помощью библиотеки re проведем отчистку текста от знаков препинания.

In [8]:
reg_expr=[]

for text in df_slice['text']:
    regular_expression = re.sub(r'[^a-zA-Z]',' ',text)
    reg_expr.append(' '.join(regular_expression.split()))
    
df_slice['reg_expr'] = reg_expr

In [9]:
df_slice#добавили в датасет новый столбец

,text,toxic,reg_expr
0,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
1,Hey... what is it..\n@ | talk .\nWhat is it......,1,Hey what is it talk What is it an exclusive gr...
2,"Bye! \n\nDon't look, come or think of comming ...",1,Bye Don t look come or think of comming back T...
3,You are gay or antisemmitian? \n\nArchangel WH...,1,You are gay or antisemmitian Archangel WHite T...
4,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,FUCK YOUR FILTHY MOTHER IN THE ASS DRY
...,...,...,...
37720,"""\n\n This is wrong \n\n""""Change in playoff fo...",0,This is wrong Change in playoff format The pla...
37721,"The article Fisher, Jonothan has been speedily...",0,The article Fisher Jonothan has been speedily ...
37722,"""\nYour bizarre fascination with me is disturb...",0,Your bizarre fascination with me is disturbing...
37723,Minnie Minoso's first game with White Sox (hom...,0,Minnie Minoso s first game with White Sox home...


### Токенизация и лемматизация

С помощью библиотеки nltk произведем токенизацию и лемматизацию слов.

In [10]:
lemmatizer = WordNetLemmatizer()

def lemmatization(data):
    lemm_texts_list = []
    for text in data:
        word_list = nltk.word_tokenize(text)
        text_lem = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
        lemm_texts_list.append(text_lem)
    return lemm_texts_list

In [11]:
df_slice['lem_text'] = pd.Series(lemmatization(df_slice['reg_expr']))

In [12]:
df_slice

,text,toxic,reg_expr,lem_text
0,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
1,Hey... what is it..\n@ | talk .\nWhat is it......,1,Hey what is it talk What is it an exclusive gr...,Hey what is it talk What is it an exclusive gr...
2,"Bye! \n\nDon't look, come or think of comming ...",1,Bye Don t look come or think of comming back T...,Bye Don t look come or think of comming back T...
3,You are gay or antisemmitian? \n\nArchangel WH...,1,You are gay or antisemmitian Archangel WHite T...,You are gay or antisemmitian Archangel WHite T...
4,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,FUCK YOUR FILTHY MOTHER IN THE ASS DRY,FUCK YOUR FILTHY MOTHER IN THE ASS DRY
...,...,...,...,...
37720,"""\n\n This is wrong \n\n""""Change in playoff fo...",0,This is wrong Change in playoff format The pla...,This is wrong Change in playoff format The pla...
37721,"The article Fisher, Jonothan has been speedily...",0,The article Fisher Jonothan has been speedily ...,The article Fisher Jonothan ha been speedily d...
37722,"""\nYour bizarre fascination with me is disturb...",0,Your bizarre fascination with me is disturbing...,Your bizarre fascination with me is disturbing...
37723,Minnie Minoso's first game with White Sox (hom...,0,Minnie Minoso s first game with White Sox home...,Minnie Minoso s first game with White Sox home...


Произведем разделение датасета на тренировочный и тестовый сет.

In [13]:
X = df_slice['lem_text']
y = df_slice['toxic']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

Скачаем словарь стоп-слов в англ.языке.

In [14]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Формируем наши матрицы признаков, с уже очищенными текстами. 

In [15]:
tf_idf = count.fit_transform(X_train)
tf_idf_test = count.transform(X_test)

## Обучение

### DecisionTreeClassifier

In [27]:
model = DecisionTreeClassifier(random_state=42,max_depth=39, min_samples_leaf=5)
model.fit(tf_idf,y_train)
predict = model.predict(tf_idf_test)
score = f1_score(y_test,predict)
score

0.7513340448239062

### LogisticRegression

In [16]:
model = LogisticRegression(random_state=42)
model.fit(tf_idf,y_train)
predict = model.predict(tf_idf_test)
score = f1_score(y_test,predict)
score

0.8675034867503486

### CatBoostClassifier

In [20]:
model = CatBoostClassifier(task_type='CPU', iterations = 1000, eval_metric='TotalF1',verbose=100)
model.fit(tf_idf,y_train)
predict = model.predict(tf_idf_test)
score = f1_score(y_test,predict)
score

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

Learning rate set to 0.041688
0:	learn: 0.6773910	total: 950ms	remaining: 15m 49s
100:	learn: 0.8231170	total: 1m 9s	remaining: 10m 21s
200:	learn: 0.8535910	total: 2m 15s	remaining: 9m
300:	learn: 0.8697478	total: 3m 20s	remaining: 7m 46s
400:	learn: 0.8835590	total: 4m 26s	remaining: 6m 37s
500:	learn: 0.8939617	total: 5m 32s	remaining: 5m 30s
600:	learn: 0.9012714	total: 6m 40s	remaining: 4m 25s
700:	learn: 0.9065968	total: 7m 48s	remaining: 3m 19s
800:	learn: 0.9130428	total: 8m 56s	remaining: 2m 13s
900:	learn: 0.9173928	total: 10m 1s	remaining: 1m 6s
999:	learn: 0.9221125	total: 11m 6s	remaining: 0us


0.8674433941525611

## Выводы

Обратила внимание, что при использовании балансировки классов в моделях,результат метрики немного становится лучше. В нашем случае, в датасете был сильный дисбаланс классов - порядка 20/80,но я искусственно привела его к 50/50.

Большее время и ресурсы затрачиваются именно на лемматизацию текстов и обработку матриц признаков моделями.
Логистическая регрессия оказалась самой быстрой в исполнении, однако если подавать ей полный датасет, то она вроде и выдаст значение метрики, но с ошибкой STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Выводы по моделям особо какие то не могу сделать.. модели отличаются подбором параметров, количеством затраченного времени на обработку матрицы признаков, подходом.. Думаю, выбор модели будет зависить от бизнес-задачи и от того, какая цель в итоге проекта будет стоять.  